根据我们之前学习的内容，我们今天就来做些有趣的数据分析。 用小项目来带动真实的 Numpy 运用过程。

20年开始的新冠肺炎席卷全球，我们能从肺炎数据中发现什么规律呢？我提前给你准备了从 2020 年 1 月 22 日到 2020 年 7 月 27 日的数据。

### 数据一览
这份数据是 Kaggle 上的公开数据集( https://www.kaggle.com/imdevskp/corona-virus-report/version/166 )， 我已经提前给你准备好了，你在下面的代码编辑框中，可以通过 covid["data"] 和 covid["header"] 获取到这些数据的信息。

In [ ]:
# 数据标题
covid["header"]
# 数据
print(covid["data"][:5]) # 数据太多了，我先打 5 行
# print(covid["data"]) # 如果你实在想看全部，就执行这一行
# 日期数据
print(covid["date"][:5]) # 数据太多了，我先打 5 行
# print(covid["date"]) # 如果你实在想看全部，就执行这一行

如果你不方便下载，我这里也有一份同样的，有兴趣你可以自己下载下来 ，用下面的代码在你本地加载数据。

In [1]:
with open(r"C:\工作任务\AIUI\培训\莫烦NumPy\archive\day_wise.csv", "r", encoding="utf-8") as f:
    data = f.readlines()

covid = {
    "date": [],
    "data": [],
    "header": [h for h in data[0].strip().split(",")[1:]]
} # covid 有3个字段：date/data/header
for row in data[1:]:
    split_row = row.strip().split(",")
    covid["date"].append(split_row[0])
    covid["data"].append([float(n) for n in split_row[1:]]) #以float形式存储


了解了数据的大概样子，那么来使用你所学到的 Numpy 技能，我们来分析下面这几个问题。

获取 2020 年 2 月 3 日的所有数据
2020 年 1 月 24 日之前的累积确诊病例有多少个？
2020 年 7 月 23 日的新增死亡数是多少？
从 1 月 25 日到 7 月 22 日，一共增长了多少确诊病例？
每天新增确诊数和新恢复数的比例？平均比例，标准差各是多少？
画图展示新增确诊的变化曲线
画图展示死亡率的变化曲线

### 某日所有数据
#### 获取 2020 年 2 月 3 日的所有数据

这个理解起来比较简单，但是在使用 Numpy 获取到特定数据的时候，我们得搞清楚数据的索引是什么。在 Numpy 中，我们用来索引的大概率是数字， 所以我们得搞清楚对应的数字标号是什么。因为我在为你准备的 covid 数据中，把日期和标题都用 Python 列表给你装起来了，你可以通过列表的 .index("xxx") 功能来获取到它的索引。

我们需要思考的是：找到这一天对应的索引。还好我已经把日期都用列表给存储起来了。我们看看日期的存储方式。 将日期转换成 date_idx 之后，我们就能用这个 index 来获取到那一天的全部数据了。

In [2]:
print("日期列表摘取：", covid["date"][:4])

date_idx = covid["date"].index("2020-02-03")
print("日期->索引转换：", date_idx)

import numpy as np
data = np.array(covid["data"])

for header, number in zip(covid["header"], data[date_idx]):
    print(header, ":", number)

日期列表摘取： ['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25']
日期->索引转换： 12
Confirmed : 19887.0
Deaths : 426.0
Recovered : 604.0
Active : 18857.0
New cases : 3100.0
New deaths : 64.0
New recovered : 145.0
Deaths / 100 Cases : 2.14
Recovered / 100 Cases : 3.04
Deaths / 100 Recovered : 70.53
No. of countries : 25.0


至于，这些人数为什么会是一个小数呢？因为 Numpy 在存储一批数据的时候，它会把数据都存成一个模式，在这套数据中，我们都是以小数（float64）的形式存储的， 所以你才会看到有小数点。

### 累积确诊
#### 2020 年 1 月 24 日之前的累积确诊病例有多少个？

有了上一个任务的基础，这个任务我们就更加细致地去获取某个具体的数值，我们确定完日期（row）的 index 之后，在确定标题 （column）的 index。 在这个案例中，我们要确定累积确诊数（Conformed）的索引。

In [3]:
row_idx = covid["date"].index("2020-01-24")     # 获取日期索引
column_idx = covid["header"].index("Confirmed") # 获取标题的索引
confirmed0124 = data[row_idx, column_idx]
print("截止 1 月 24 日的累积确诊数：", confirmed0124)

截止 1 月 24 日的累积确诊数： 941.0


### 新增死亡
#### 2020 年 7 月 23 日的新增死亡数是多少？

这还是一个在 Numpy Array 中选取数据的过程，我们要搞清楚具体的索引是什么。上面我们已经找到了确诊数的标题（Conformed），现在你来找一下哪个标题对应着新增死亡数？

没错！就是New deaths, 我们再按上面的流程把这个数据给拿出来。

In [4]:
row_idx = covid["date"].index("2020-07-23")     # 获取日期索引
column_idx = covid["header"].index("New deaths") # 获取标题的索引
result = data[row_idx, column_idx]
print("截止 7 月 23 日的新增死亡数：", result)

截止 7 月 23 日的新增死亡数： 9966.0


### 总增长数
#### 从 1 月 25 日到 7 月 22 日，一共增长了多少确诊病例？

获取数据都是小儿科，我们来点真正的数据分析。要基于原有的数据，进行多次加工，来获取新的结论和结果。在这个任务中，我们要进行一次累和的计算。

In [5]:
row1_idx = covid["date"].index("2020-01-25")
row2_idx = covid["date"].index("2020-07-22")
new_cases_idx = covid["header"].index("New cases")

# 注意要 row1_idx+1 得到从 01-25 这一天的新增
# row2_idx+1 来包含 7 月 22 的结果
new_cases = data[row1_idx+1: row2_idx+1, new_cases_idx]
overall = new_cases.sum()
print("共新增：", overall)

共新增： 15247309.0


有的同学肯定会问了，我们不是有累积确诊的个数嘛，我也能用累计确诊的那一 column 来计算出这段时间内的确诊病例数呀。 好的，我们就来按这种方法计算一下。

In [6]:
confirm_idx = covid["header"].index("Confirmed")
confirmed = data[:, confirm_idx]
overall2 = confirmed[row2_idx] - confirmed[row1_idx]
print("共新增：", overall2)

共新增： 15226291.0


奇怪了，为什么两种计算结果对应不上呢？我大胆猜想一下，可能是因为有人感染了两次或多次，导致新增的数据中重复计算了。 所以用新增数据来算的话，比累积确诊的方式来计算的要多。

我们还能具体分析看看，具体是哪天的数据对不上，详细输出一下。

In [7]:
confirmed = data[:, confirm_idx]
new_cases = data[:, new_cases_idx]

for i in range(row1_idx, row2_idx+1):
    diff = new_cases[i] - (confirmed[i] - confirmed[i-1]) 
    if diff != 0:
        print("date index:", i, ";差异：", diff)

date index: 62 ;差异： 15.0
date index: 81 ;差异： 21.0
date index: 87 ;差异： 1.0
date index: 91 ;差异： 2188.0
date index: 93 ;差异： 10034.0
date index: 95 ;差异： 1.0
date index: 97 ;差异： 105.0
date index: 98 ;差异： 2512.0
date index: 101 ;差异： 167.0
date index: 103 ;差异： 1.0
date index: 106 ;差异： 1583.0
date index: 107 ;差异： 1480.0
date index: 109 ;差异： 9.0
date index: 110 ;差异： 57.0
date index: 111 ;差异： 20.0
date index: 112 ;差异： 165.0
date index: 113 ;差异： 1.0
date index: 118 ;差异： 209.0
date index: 120 ;差异： 104.0
date index: 124 ;差异： 372.0
date index: 125 ;差异： 220.0
date index: 132 ;差异： 766.0
date index: 133 ;差异： 1.0
date index: 149 ;差异： 148.0
date index: 158 ;差异： 719.0
date index: 163 ;差异： 1.0
date index: 175 ;差异： 5.0
date index: 176 ;差异： 3.0
date index: 181 ;差异： 110.0


### 确诊恢复比例
#### 每天新增确诊数和新恢复数的比例？平均比例，标准差各是多少？

这一个任务，我们需要再计算计算，会要使用到 Numpy 的批量计算功能，首先确定要拿取到的数据是什么，然后再对这些数据进行计算。 按要求，我们要拿到新增确诊数（New cases），和新的恢复数（New recovered）,然后再相除，最后再计算平均值和标准差。

In [8]:
new_cases_idx = covid["header"].index("New cases")
new_recovered_idx = covid["header"].index("New recovered")

# 比例
ratio = data[:, new_cases_idx] / data[:, new_recovered_idx]
print("比例样本：", ratio[:5])

比例样本： [         nan  49.5         47.83333333 164.33333333  52.61538462]


C:\Users\OS\Anaconda3\envs\rl_env\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


咦奇怪了，我什么第一个是 nan 呢？nan 在 numpy 中表示的是 Not a Number， 说明计算有问题。我们单独打印一下，看看为什么出现 nan。

In [9]:
print(data[0, new_cases_idx])
print(data[0, new_recovered_idx])

0.0
0.0


原来是因为 new_recovered_idx 的第一个位置数据为 0，任何数除以 0 是不成立的，所以才会把这个计算给出一个 nan 的结果。 你看做数据分析还是挺有意思吧，会出现各种意想不到的状况。然后我们把 new_recovered 为零的数都剔除掉。

In [10]:
not_zero_mask = data[:, new_recovered_idx] != 0 #mask的使用
ratio = data[not_zero_mask, new_cases_idx] / data[not_zero_mask, new_recovered_idx]

# 平均比例, 标准差
ratio_mean = ratio.mean()
ratio_std = ratio.std()
print("平均比例：", ratio_mean, "；标准差：", ratio_std)

平均比例： 7.049556348053241 ；标准差： 19.094025710450307


得到计算结果后，我们还是能明显发现，被治好的人的确比新增的要多，这就说明，我们自己的存活率还是挺高的。

### 可视化数据
#### 画图展示新增确诊的变化曲线

有时候，光秃秃的数字并不能生动表现这些数字背后的规律。我们还可以把它画出来。至于如何画，我给你做了一个 draw_line() 的函数，这个函数的背后是 Python 的画图工具 Matplotlib. 不过在这个教程中，并不是重点，我会在之后的画图教程中详细介绍。

现在，我们就来做新增确诊的可视化数据吧。

In [11]:
new_cases_idx = covid["header"].index("New cases")
draw_line(data[:, new_cases_idx])

NameError: name 'draw_line' is not defined

不画不知道，一画出来就吓一跳，在前半部分的时候，确诊数还在缓慢增长，但是在 50 天左右的时候，发现新增确诊数陡增。我们看看第 50 天是哪个日期。 新冠在前期还是传播得十分迅速的。

In [12]:
print(covid["date"][50])

2020-03-12


### 画图展示死亡率的变化曲线

画这个图也非常简单，我们只要找到对应的 column 就行。我只是觉得这个数据比较有趣，想看看人类对于一个陌生疾病的斗争历史。 因为在前期，我们对新冠还不了解的时候，可能死亡率还是比价高的，但是随着了解的加深，政策的颁布，疫苗的发布，死亡率应该都会有变化。

In [13]:
death_ratio_idx = covid["header"].index("Deaths / 100 Cases")
draw_line(data[:, death_ratio_idx])

NameError: name 'draw_line' is not defined

你看，结果也是能在图上看到的，大概在 100 天左右的时候，死亡率开始下降了，100 个人中死 7 个，逐渐变成死 4 个。可见当时各种政策的颁布还是有效果的。

### 总结
数据分析是一件非常有趣的事情。我之前租房子，还用爬虫做过悉尼各地区的房价分析，为我的租房开销节省了不少钱。其中我就用到了 numpy 来管理我爬取到的数据。